# Imports

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle

account="account1"
dataName="sampleData1"

## Read Data & Clean

In [2]:
# reading csv file  
rawDF=pd.read_csv("../delete/SampleCrimeData.csv") 

#This is specific to this use case. Modify it as per need
rawDF.replace(',','', regex=True, inplace=True)

## Preview Data

In [3]:
rawDF.head()

,Year,Population1,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate
0,1997,267783607,1636096,611.0,498534,186.2,1023201,382.1,11558475,4316.30,2460526,918.8,7743760,2891.80,1354189,505.7
1,1998,270248003,1533887,567.6,447186,165.5,976583,361.4,10951827,4052.50,2332735,863.2,7376311,2729.50,1242781,459.9
2,1999,272690813,1426044,523.0,409371,150.1,911740,334.3,10208334,3743.60,2100739,770.4,6955520,2550.70,1152075,422.5
3,2000,281421906,1425486,506.5,408016,145.0,911706,324.0,10182584,3618.30,2050992,728.8,6971590,2477.30,1160002,412.2
4,20015,285317559,1439480,504.5,423557,148.5,909023,318.6,10437189,3658.10,2116531,741.8,7092267,2485.70,1228391,430.5


## Extract Keys 

In [4]:
originalColumns=rawDF.columns

def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min(),x.max()])

originalMinMax=rawDF.apply(minMax)


### Preview Keys

In [5]:
print(type(originalColumns))
print(originalColumns.values.tolist())
originalMinMax

<class 'pandas.core.indexes.base.Index'>
['Year', 'Population1', 'Violent\ncrime2', 'Violent \ncrime \nrate', 'Robbery', 'Robbery \nrate', 'Aggravated \nassault', 'Aggravated \nassault rate', 'Property \ncrime', 'Property \ncrime \nrate', 'Burglary', 'Burglary \nrate', 'Larceny-\ntheft', 'Larceny-\ntheft rate', 'Motor \nvehicle \ntheft', 'Motor \nvehicle \ntheft \nrate']


,Year,Population1,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate
min,1997,267783607,1153022,361.6,322905,101.3,1023201,229.2,10019601,2450.70,1515096,468.9,5638455,1745.00,1100472,215.4
max,20156,323127513,1636096,611.0,498534,186.2,976583,382.1,9882212,4316.30,2460526,918.8,7743760,2891.80,959059,505.7


## Scale the columns 

In [6]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(rawDF), columns=rawDF.columns)

## Preview result

In [7]:
df_scaled.head()

,Year,Population1,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate
0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.000055,0.044529,0.788420,0.825982,0.707634,0.756184,0.842732,0.864617,0.833313,0.858598,0.864833,0.876417,0.825465,0.858476,0.833068,0.842232
2,0.000110,0.088668,0.565176,0.647153,0.492322,0.574794,0.623981,0.687377,0.629025,0.693021,0.619446,0.670149,0.625593,0.702564,0.697156,0.713400
3,0.000165,0.246428,0.564021,0.580994,0.484607,0.514723,0.623866,0.620013,0.621950,0.625858,0.566828,0.577684,0.633227,0.638559,0.709033,0.677919
4,0.992235,0.316818,0.592990,0.572975,0.573094,0.555948,0.614815,0.584696,0.691907,0.647191,0.636150,0.606579,0.690547,0.645884,0.811506,0.740958


## Replace Column names 

In [8]:
#df_scaled.columns = [''] * len(df_scaled.columns)
col_list = ['dx' + str(x) for x in range(1,1+len(df_scaled.columns))]
df_scaled.columns = col_list

columnMapping = pd.DataFrame([col_list],columns=originalColumns) 

columnMapping.head()

,Year,Population1,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate
0,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dx16


# Save Data and Keys

In [10]:
df_scaled.to_csv('../Bank/'+account+'/'+dataName+'.csv', index=False)
df_scaled.to_json('../Bank/'+account+'/'+dataName+'.json', orient='columns')
originalMinMax.to_csv('../AccountKeys/'+account+'_'+dataName+'-MiMa.csv', index=False)
originalMinMax.to_json('../AccountKeys/'+account+'_'+dataName+'-MiMa.json',orient='columns') # orient= columns  split
columnMapping.to_json('../AccountKeys/'+account+'_'+dataName+'-CoMap.json',orient='records') # orient= columns  split records

with open('../AccountKeys/'+account+'_'+dataName+'.txt', "wb") as fp:   #Pickling
    pickle.dump(originalColumns.values.tolist(), fp)

## Test saved data

In [11]:
df_scaledRead=pd.read_csv('../Bank/'+account+'/'+dataName+'.csv') 
originalMinMaxRead=pd.read_csv('../AccountKeys/'+account+'_'+dataName+'-MiMa.csv') 
with open('../AccountKeys/'+account+'_'+dataName+'.txt', "rb") as fp:   # Unpickling
     originalColumnsRead = pickle.load(fp)
df_scaledRead.columns = originalColumnsRead

In [12]:
df_scaledRead.head()

,Year,Population1,Violent crime2,Violent crime rate,Robbery,Robbery rate,Aggravated assault,Aggravated assault rate,Property crime,Property crime rate,Burglary,Burglary rate,Larceny- theft,Larceny- theft rate,Motor vehicle theft,Motor vehicle theft rate
0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.000055,0.044529,0.788420,0.825982,0.707634,0.756184,0.842732,0.864617,0.833313,0.858598,0.864833,0.876417,0.825465,0.858476,0.833068,0.842232
2,0.000110,0.088668,0.565176,0.647153,0.492322,0.574794,0.623981,0.687377,0.629025,0.693021,0.619446,0.670149,0.625593,0.702564,0.697156,0.713400
3,0.000165,0.246428,0.564021,0.580994,0.484607,0.514723,0.623866,0.620013,0.621950,0.625858,0.566828,0.577684,0.633227,0.638559,0.709033,0.677919
4,0.992235,0.316818,0.592990,0.572975,0.573094,0.555948,0.614815,0.584696,0.691907,0.647191,0.636150,0.606579,0.690547,0.645884,0.811506,0.740958


# Preffered way to share data

Step 1: Gor to "Bank" folder/directory  
Step 2: Check if the account01/02... folder is created and has the data  
Step 3: If you want it to publish it for public usage, then submit to Git repository, and raise a merge request  
Step 4: For private usage you can transmit the data any way you feel comfortable with,  
Step 5: Do not commit data in "AccountKeys" unless you intend to make data public  
Step 6: Delete data from "delete" folder, before commiting/pushing data to git  